<a href="https://colab.research.google.com/github/dvirbz/NLP2025/blob/main/Dvir_and_Amit_NLP_HW4P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir=rtl>

# עיבוד שפה טבעית (20225211) 2024-2025 - תרגיל 4 חלק 1 (5 נק')

שלום לכולםן!

בחלק הזה של תרגיל 4 נצבור קצת נסיון מעשי בעבודה עם מודלי שפה מסוג רובוטריק (transformer).
נתמקד במודל מסוג **אוטורגרסיבי**, כלומר חוזה-פני-עתיד, כלומר מתבונן-רק-שמאלה.
ספציפית, נסתכל על המודל GPT-2, גרסה מ-2019 של המודל של OpenAI. מבחינה אלגוריתמית, אין הרבה הבדל בינו לבין GPT-4.5 הנוכחי אלא רק מבחינת סדר-הגודל של כמות הפרמטרים וכמות הדאטא עליו אומן. מה שכן, זה לא מודל שאומן למטרות שיח ובינו לבין ג'יפיטפוט (ChatGPT) כן יש הבדלים מהותיים.

בסיום העבודה, הגישו **קובץ pdf** הכולל את כל הפלטים מכל התאים. דרך אפשרית אחת להשיג קובץ pdf היא על-ידי הדפסת הדף (מתוך colab, לא מהדפדפן) ולבחור "מדפסת" ישירה ל-pdf.

---

**נא לוודא שכל המידע הדרוש מופיע בתדפיס ה-pdf --- לא יתקבלו ערעורים על-בסיס חיתוך. נא לא לחרוג ממבנה התאים ולא לענות במקומות שאינם מוגדרים לכך.**
**אין למחוק הערות שמסמנות את מיקומי קטעי הקוד שלכםן ואין לערוך קוד שנמצא מחוץ לתחומים המוגדרים. סעיפים שחורגים מהנחיה זו לא ייבדקו.**

---

ניתן להגיש בצוותים של עד שלוש.ה סטודנטיםות. לא יהיו הנחות לצוותים קטנים. אפשר לחלק את פתרון התרגיל בין חברי.ות הצוות כרצונכםן. אין להסתכל על קוד של צוותים אחרים ואין להקריא קוד לצוותים אחרים.

**אין להשתמש ברכיב ג'מיני Gemini או כל סייען מתקדם אחר.**

מועד ההגשה הוא דקה לפני תחילת השיעור ב-13/1. לא יינתנו הארכות.

בהצלחה,

-- יובל
</div>

<div dir=rtl>

## התקנות

דבר ראשון, נתקין את החבילות הנדרשות. ל-`datasets` ו-`tokenizers` שהכרנו מתרגיל 2 מצטרפת `transformers`, גם היא מבית האגינגפייס, שבעזרתה נוכל לטעון את המודל המאומן ואת הטוקנייזר שלו ולהשתמש בהם על טקסט חופשי.
</div>

In [2]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


<div dir=rtl>

נייבא חבילות ומודולים.
הרובוטריקים של האגינגפייס יודעים לדבר עם `torch` וכיוון שנוח לנו נעבוד איתו לצרכי עיבוד נוסף של הפלטים.

</div>

In [3]:
import numpy as np
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn.functional import softmax, log_softmax
import torch

<div dir=rtl>

נטען באמצעות `datasets` את אסופת סיפורי הילדים שעבדנו איתה כדי לאמן מודל שיכוני מילים בשבוע 7. היום לא ממש נאמן כלום, אבל זה טוב שיש משפטים מן המוכן במקום להמציא.

</div>

In [4]:
stories = datasets.load_dataset("deven367/babylm-100M-children-stories")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/661 [00:00<?, ?B/s]

(…)-00000-of-00001-13f0a33230d64dd9.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

(…)-00000-of-00001-0177e6dbdee45eef.parquet:   0%|          | 0.00/905k [00:00<?, ?B/s]

(…)-00000-of-00001-0b94257b623049e5.parquet:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/76758 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/5996 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7959 [00:00<?, ? examples/s]

<div dir=rtl>

כאמור, אנחנו נעבוד עם מודל [GPT-2](https://huggingface.co/openai-community/gpt2) (הקישור מוביל לדף הסבר באתר האגינגפייס).
צריך לטעון בנפרד את המודל ואת ה**טוקנייזר שלו** שאומן בשיטת קידוד זוגות בתים, או BPE, אותה למדנו בשבוע 8.

שימו לב שבניגוד לתרגיל הקודם, הפעם לא נאמן כלום ולא נגריל כלום ולכן לא צריך לקבע זרע אקראיות.

</div>

In [5]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

<div dir=rtl>

פונקציית הייצוג של מודל בחבילת `transformers` נותנת פרטים על הארכיטקטורה של המודל בצורה מקוננת ונוחה.

**1. ענו על השאלות הבאות בהסתמך על הפלט של שני התאים הבאים (1 נק'):**
1. (0.25) מהו גודל אוצר המילים של המודל?
1. (0.25) `ModuleList` מתאר רשימת מודולים לפי סדר הפעלתם. מה שונה באופן סידור המודולים של בלוק השכבה ב-GPT-2 (כלומר, החלק שמשתכפל בהתאם למספר השכבות) מזה שלמדנו בכיתה? אין צורך להתבונן בתוך מרכיבי המודולים עצמם (כלומר ניתן להניח שמודול עם Attention בשם הוא בלוק צומי עצמי, למשל).
1. (0.5) שכבת ה-MLP של המודל משתמשת בפונקציית אקטיבציה שלא למדנו עליה. חקרו אודותיה מעט וענו מהי התכונה העיקרית שמבדילה אותה משלוש הפונקציות שאנחנו מכירים (בפחות מחמש מילים).

<hr>

**תשובות: (נא לערוך את התא הטקסטואלי)**
1. 50257

2. ניתן לראות כי בתוך ModuleList נמצאים 12 בלוקים של GPT2 וכן בתוך כל אחד מהם יש שכבת נורמליזציה, שכבת צומי עצמי שכבת נורמליזציה וGPT2MLP אשר מורכב משתי שכבות קונבולציה חד ממדיות אקטיבציה (GELU) ונישור (dropout) לכן ההבדל הינו ראש הMLP שלא נכח בראש הצומי העצמי וכן גם שכבות הנורמליזציה

3. GELU - גזירה ומוכלת בתחום $[0,\infty]$
</div>

In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
tokenizer

GPT2TokenizerFast(name_or_path='openai-community/gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

<div dir=rtl>

נרענן את זכרוננו לגבי איך מחלצים את הטקסט מתוך הדאטא.

</div>

In [8]:
story1 = list(stories['train'])[1]['text']
print(story1)

HIGH above the city, on a tall column, stood the statue of the Happy Prince.  He was gilded all over with thin leaves of fine gold, for eyes he had two bright sapphires, and a large red ruby glowed on his sword-hilt.


<div dir=rtl>

ועכשיו נבדוק איך הטוקנייזר מייצג אותו. הפונקציה הראשונה שנקרא לה תיצור עבורנו אצווה (batch) בפורמט פייטורץ (`pt`) שתכיל, בין היתר, את האלמנטים הבאים:
* קריאה לפונקצית `tokens()` תחזיר את המחרוזות אליהן התפצל הטקסט כדי להיכנס לאוצר המילים של המודל. BPE הוא טוקנייזר "תת-מילי" subword, כלומר יש מרכיבים שאינם מהווים מילים שלמות וזאת כדי שגם למילים נדירות או לא-מוכרות יהיה ייצוג. שימו לב שיש תו מיוחד שמוצג כאן כ-G עם נקודה מעל, תפקידו לסמן **רווח** והוא לרוב יתחבר למילה שאחריו.
* `input_ids` הם הטוקנים שיצר הטוקנייזר בצורתם הגולמית כאינדקסים מתוך אוצר המילים (כך, למשל, ניגש לשיכונים שלהם). אינטואיטיבית, נסו להעריך אם יש משמעות כלשהי לאינדקסים.

**2. ענו על השאלות הבאות (1):**
1. (0.5) מה מסמנים האלמנטים שמחזירה הפונקציה `word_ids()`, ובאיזה סיטואציה במיוחד הם חשובים? חישבו על הבעיות שלמדנו עד היום בקורס.
1. (0.5) בחרו עוד אלמנט בפלט של הטוקנייזר והסבירו את תפקידו. אפשר להדגים במקום המיועד לכך בתא הבא.

<hr>

**תשובות: (נא לערוך את התא הטקסטואלי)**

1. האלמנטים שמוחזרים הינם מיקומי השיכונים ביחס למילה מהם הם הגיעו / נוצרו ,   ביחס למיקום המילה במשפט. מיקומים אלו יכולים לעזור בסיטואציות בהן נרצה ליצור positional embeddings או להתחשב במיקום השיכון ביחס למשפט.

2. input_ids הינם אלמנטים המייצגים את מיקום המילה / שיכון ביחס לאוצר המילים של הטוקנייזר, מיקומים אלו יהיו האינפוטים לטרנספורמר (או יותר נכון לembeddor) ולכן נצטרך אותם על מנת להמיר מילים באצווה לשיכונים עצמם.

</div>

In [78]:
input_toks = tokenizer.batch_encode_plus([story1], return_tensors='pt')
tok_texts = input_toks.tokens()
print(' '.join(tok_texts[:10]))
print(len(input_toks.input_ids[0]))
print(input_toks.input_ids[0][:10])
print(input_toks.word_ids()[:10])
### 2.2 One more element: ###
#print(dir(input_toks), input_toks)

for k,v in tokenizer.vocab.items():
  if (v < 322 and v > 318) or v < 20 or v==50256:
    print(f"{k} id is {v}")


H IGH Ġabove Ġthe Ġcity , Ġon Ġa Ġtall Ġcolumn
58
tensor([   39, 18060,  2029,   262,  1748,    11,   319,   257,  7331,  5721])
[0, 0, 1, 2, 3, 4, 5, 6, 7, 8]
im id is 320
/ id is 14
- id is 12
3 id is 18
) id is 8
am id is 321
4 id is 19
& id is 5
, id is 11
* id is 9
$ id is 3
+ id is 10
" id is 1
( id is 7
. id is 13
# id is 2
0 id is 15
2 id is 17
1 id is 16
% id is 4
! id is 0
' id is 6
Ġon id is 319
<|endoftext|> id is 50256


<div dir=rtl>

כעת נפעיל את המודל עצמו באמצעות קריאה ישירה אליו על המבנה המפורק (שתי כוכביות פייתוניות) של פלט הטוקנייזר.
הקריאה הזו תעביר את הטוקנים דרך כל הרובוטריק, מהשיכונים ועד שכבת החיזוי, ותיתן לנו את תוצרי כל אחד משלבי החיזוי של GPT-2 **כמודל שפה**.

הפלט מגדיר מבנה די מסובך (אפשר לראות תיעוד מלא באתר הדוקומנטציה או באמצעות קריאה למודל עם סימן שאלה אחריו בתא קוד במחברת).
ניכנס ישר למה שמעניין אותנו - הלוג'יטים (logits), שהם הציונים הגולמיים של כל הטוקנים באוצר המילים בכל אחד משלבי החיזוי, כלומר על כל תחילית של הקלט.
כך האיבר הרביעי במבנה הלוג'יטים ייתן את כל התחזיות עבור כל הטוקנים האפשריים בהינתן ארבעת הטוקנים הראשונים של הטקסט, כלומר את מרחב התחזיות לטוקן החמישי. לאחר שנעביר את הווקטור הזה דרך סופטמקס, נקבל את התחזיות האלה כ**התפלגות**.

</div>

In [10]:
generative_outputs = model(**input_toks)
print(generative_outputs[0].shape)
gen_out_logits = generative_outputs[0][0]
print(len(gen_out_logits))
print(len(gen_out_logits[3]))
print(float(gen_out_logits[3][1748]))
print(float(gen_out_logits[3][0]))

torch.Size([1, 58, 50257])
58
50257
-79.91033172607422
-87.89537811279297


In [11]:
softmaxed_outputs = softmax(gen_out_logits[3], dim=0)  # specify a dimension or an annoying warning comes up
print(float(softmaxed_outputs[1748]))
print(torch.argmax(softmaxed_outputs), softmaxed_outputs[4417], [(k,v) for k,v in tokenizer.vocab.items() if v == 4417 or k =="Ġcity"])

0.005037951283156872
tensor(4417) tensor(0.0524, grad_fn=<SelectBackward0>) [('Ġsurface', 4417), ('Ġcity', 1748)]


<div dir=rtl>

נראה שהמודל מעניק למילה האמיתית, city, הסתברות של כחצי אחוז להיות המילה הבאה.
זה לא מעט בהינתן גודל אוצר המילים הכללי, אבל האם יש מילים שהוא מעדיף?

**3. ענו (0.5):**
1. (0.25) כמה מילים מעדיף המודל על-פני city?
1. (0.25) מהי המילה לה הוא מעניק את ההסתברות הגבוהה ביותר? (**לא האינדקס: המילה עצמה**)

</div>

In [109]:
### Answers for 3 (edit where necessary) ###

### Answer for 3.1 ###
city_prob = float(softmaxed_outputs[1748])
print(f"Number of words more probable than city: {len([prob for prob in softmaxed_outputs if prob > city_prob])}")

### Answer for 3.2 ###

def best_token_id(sm_output) -> int:
  best_5_ids = torch.argsort(sm_output, descending=True)[:20]
  print(f"{tokenizer.convert_ids_to_tokens(best_5_ids)=}")
  best_input_id = best_5_ids[0]
  print(f"{best_input_id=}")
  return best_input_id


def token_from_index(idx: int) -> str:
  '''
  Hint: look at the tokenizer
  '''
  return tokenizer.convert_ids_to_tokens([idx])[-1]



best_in_outputs = best_token_id(softmaxed_outputs)
print(f"Most probable word: {token_from_index(best_in_outputs)}")

Number of words more probable than city: 13
tokenizer.convert_ids_to_tokens(best_5_ids)=['Ġsurface', 'Ġground', 'Ġwater', 'Ġhorizon', 'Ġtop', 'Ġlevel', 'Ġsea', 'Ġclouds', 'Ġearth', 'Ġocean', 'Ġrim', 'Ġhead', 'Ġthreshold', 'Ġcity', 'Ġcenter', 'Ġline', 'Ġhigh', 'Ġaverage', 'Ġbar', 'Ġfloor']
best_input_id=tensor(4417)
Most probable word: Ġsurface


<div dir=rtl>

וכך נוכל לראות את ההסתברויות של כל המילים ברצף. שימו לב שיש הסתברויות מאוד גבוהות להמשכי המילים הנכונים כאשר הטוקנייזר חותך אותן, למשל בתוך המילה sapphires:

</div>

In [55]:
word_probs = []
for loc, tok_idx in enumerate(input_toks.input_ids[0]):
  if loc < 1: continue  # think why!
  sm_loc = softmax(gen_out_logits[loc - 1], dim = 0)
  prob = sm_loc[tok_idx]
  word_probs.append((tok_texts[loc], prob))
print('; '.join([f'{txt}: {prob:.4f}' for txt, prob in word_probs]))

IGH: 0.0000; Ġabove: 0.0000; Ġthe: 0.3064; Ġcity: 0.0050; ,: 0.1424; Ġon: 0.0086; Ġa: 0.2081; Ġtall: 0.0012; Ġcolumn: 0.0004; ,: 0.1255; Ġstood: 0.0065; Ġthe: 0.2807; Ġstatue: 0.0068; Ġof: 0.9195; Ġthe: 0.1156; ĠHappy: 0.0001; ĠPrince: 0.0061; .: 0.1924; Ġ: 0.0004; ĠHe: 0.0000; Ġwas: 0.3196; Ġg: 0.0003; ilded: 0.2881; Ġall: 0.0006; Ġover: 0.3727; Ġwith: 0.3027; Ġthin: 0.0001; Ġleaves: 0.0011; Ġof: 0.1925; Ġfine: 0.0044; Ġgold: 0.2293; ,: 0.3964; Ġfor: 0.0018; Ġeyes: 0.0001; Ġhe: 0.0039; Ġhad: 0.2253; Ġtwo: 0.0001; Ġbright: 0.0009; Ġsa: 0.0007; pp: 0.8589; h: 0.5735; ires: 0.9995; ,: 0.2797; Ġand: 0.3091; Ġa: 0.1759; Ġlarge: 0.0324; Ġred: 0.0147; Ġruby: 0.0014; Ġgl: 0.0032; owed: 0.1695; Ġon: 0.0650; Ġhis: 0.6804; Ġsword: 0.0014; -: 0.0448; h: 0.0069; ilt: 0.3827; .: 0.6618


<div dir=rtl>

כמו שהזכרנו בכיתה, קשה למצוא מטריקות טובות להערכת חילול טקסט. מה שכן אפשר לעשות זה להעריך מודלים באמצעות מידת הסבירות הכוללת שהם נותנים לטקסט נתון. המטריקה, שגם מיתרגמת לפונקציית הפסד נוחה, היא **בלבלות**, או בלעז **[perplexity](https://en.wikipedia.org/wiki/Perplexity)**.
היא הגיעה אלינו מתורת האינפורמציה, והרעיון הכללי שבה הוא לחשב כמה המודל "מופתע" מהרצף שהוא נתקל בו, לעומת מה שהוא "רגיל" אליו.

להלן נוסחאתה:

$$ PPL(\mathbf{x}) = \exp\left\{-\frac{1}{N}\sum_i^N\log P(x_i|x_{<i})\right\} $$

בכל נקודה ברצף, ככל שלוג ההסתברות גבוה יותר כך המודל "ציפה" יותר למילה האמיתית. לאחר השלילה וההעלאה מחדש בחזקה, מקבלים מדד שככל שהערך שלו **גבוה** יותר, המודל **פחות טוב** בזיהוי הרצף. באופן מדויק יותר, ככל שהבלבלות גבוהות כך הרצף פחות סביר לפי המודל.

**4. ממשו את פונקציית הבלבלות וחשבו אותה עבור הרצף שלנו (1).**

שימו לב שלוג ההסתברות אינו הלוג'יט! (מהו כן הלוג'יט? חכו לתרגיל 5.) תצטרכו לעשות כאן שימוש בפונקציה שייבאנו מבעוד מועד ביחד עם הסופטמקס (ר' לעיל) שמחשבת לוג הסתברות באופן ישיר ומהיר וללא בעיות נומריות.

כמו כן, שימו לב לשינוי הקטנטן שנאלצנו לעשות בתא הקודם כתוצאה מכך ש-GPT לא מתחיל רצפים עם טוקן מיוחד, והתאימו את הנוסחא בשאלה הזו.

</div>

In [56]:
def perplexity(logits, true_idxs) -> float:
  assert len(logits) == len(true_idxs)
  N = len(true_idxs)

  ### 4. keep going ###
  words_probs = []
  for loc, tok_idx in enumerate(input_toks.input_ids[0]):
    if loc < 1: continue
    lsm_loc = log_softmax(gen_out_logits[loc - 1], dim = 0)
    prob = lsm_loc[tok_idx]
    words_probs.append(prob)
  ppl = torch.exp(-1*sum(words_probs)/(N-1))
  return ppl

print(perplexity(gen_out_logits, input_toks.input_ids[0]))

tensor(65.2570, grad_fn=<ExpBackward0>)


<div dir=rtl>

נסבר את האוזן עם חישוב אותה המטריקה עבור המקטע הבא של הסיפור. אפשר לראות אם המודל "ציפה" לו יותר או פחות מלמקטע הראשון.

ואפשר גם סתם על טקסט חופשי, כמו בתא שאחריו.

ולראות אם GPT-2 מכיר ביטויים מפורסמים באנגלית.

</div>

In [57]:
story2 = list(stories['train'])[2]['text']
input_toks_2 = tokenizer.batch_encode_plus([story2], return_tensors='pt')
print(len(input_toks_2.tokens()))
print(' '.join(input_toks_2.tokens()))
generative_outputs_2 = model(**input_toks_2)[0][0]
print(perplexity(generative_outputs_2, input_toks_2.input_ids[0]))

68
He Ġwas Ġvery Ġmuch Ġadmired Ġindeed . Ġ ĠâĢ ľ He Ġis Ġas Ġbeautiful Ġas Ġa Ġweather cock , âĢ Ŀ Ġremarked Ġone Ġof Ġthe ĠTown ĠCoun cill ors Ġwho Ġwished Ġto Ġgain Ġa Ġreputation Ġfor Ġhaving Ġartistic Ġtastes ; ĠâĢ ľ only Ġnot Ġquite Ġso Ġuseful , âĢ Ŀ Ġhe Ġadded , Ġfearing Ġlest Ġpeople Ġshould Ġthink Ġhim Ġunp ractical , Ġwhich Ġhe Ġreally Ġwas Ġnot .
tensor(34.9774, grad_fn=<ExpBackward0>)


In [58]:
idiom = 'The rain in Spain stays mainly on the plain.'
idiom_toks = tokenizer.batch_encode_plus([idiom], return_tensors='pt')
print(' '.join(idiom_toks.tokens()))
gen_outs_idiom = model(**idiom_toks)[0][0]
perplexity(gen_outs_idiom, idiom_toks.input_ids[0])

The Ġrain Ġin ĠSpain Ġstays Ġmainly Ġon Ġthe Ġplain .


tensor(3.1091e+11, grad_fn=<ExpBackward0>)

In [59]:
true_idiom_last_tok = idiom_toks.input_ids[0][8]
print(int(true_idiom_last_tok))
idiom_sm = softmax(gen_outs_idiom[7], dim=0)
print(float(idiom_sm[true_idiom_last_tok]))
print(float(idiom_sm.max()))

most_probable_token = int(best_token_id(idiom_sm))
print(most_probable_token, token_from_index(most_probable_token))

8631
0.00023369898553937674
0.11648979038000107
tensor(2323)
2323 Ġground


<div dir=rtl>

הפעם נבקש מ-GPT-2 להשלים רצפים בלי שאנחנו נותנים אותם כחלק מהקלט.
לא יהיה כאן שום הבדל מבחינת ההתנהגות, הודות לסגירת המשולש העליון של מטריצת הצומי, אבל ככה נדמה יותר טוב את אופן הפעולה של המודל **בשלב היישום** (שפוגשים היום עם מודלי הצ'ט למיניהם): נותנים טקסט והמודל משלים.

**5. עבור שלושת הרצפים הבאים, מצאו את המילה ש-GPT-2 ישלים תחת מנגנון חילול חמדן (כלומר, את הטוקן הכי סביר) (0.5):**

</div>

In [73]:
sentences_to_complete = ['Why is the sky',
                         "You ain't seen nothing",
                         'Question: When was Barack Obama elected President of the USA? Answer:',
                         "You are on top of the"]

In [110]:
def most_prob_continuation(sentence: str) -> int:

  ### Q5 - edit here ###
  input_tokens = tokenizer.batch_encode_plus([sentence], return_tensors='pt')
  outputs = model(**input_tokens)
  logits = outputs[0][0]
  sm_outputs = softmax(logits[-1], dim=0)
  return best_token_id(sm_outputs)


for s in sentences_to_complete:
  most_prob_cont = most_prob_continuation(s)
  print(int(most_prob_cont), token_from_index(most_prob_cont))

tokenizer.convert_ids_to_tokens(best_5_ids)=['Ġblue', 'Ġso', 'Ġa', 'Ġgoing', 'Ġfalling', 'Ġreally', 'Ġblack', 'Ġthe', 'Ġfilled', 'Ġsky', '?', 'Ġin', 'Ġbeing', 'Ġabove', 'Ġwhite', 'Ġnot', 'Ġmoving', 'Ġdark', 'Ġred', 'Ġcovered']
best_input_id=tensor(4171)
4171 Ġblue
tokenizer.convert_ids_to_tokens(best_5_ids)=['Ġyet', 'Ġlike', ',', 'Ġbut', '.', 'Ġin', '."', "'", 'Ġbefore', 'Ġso', 'Ġto', 'Ġsince', 'Ġof', ".'", 'Ġyou', 'Ġthat', 'Ġfor', 'ĠI', 'Ġwrong', 'Ġnew']
best_input_id=tensor(1865)
1865 Ġyet
tokenizer.convert_ids_to_tokens(best_5_ids)=['ĠJanuary', 'ĠNovember', 'ĠMarch', 'ĠFebruary', 'ĠSeptember', 'ĠOctober', 'ĠJuly', 'ĠIn', 'ĠOn', 'ĠApril', 'ĠHe', 'ĠAugust', 'ĠJune', 'ĠDecember', 'ĠMay', 'ĠThe', 'ĠWhen', 'ĠAfter', 'Ġ9', 'Ġon']
best_input_id=tensor(3269)
3269 ĠJanuary
tokenizer.convert_ids_to_tokens(best_5_ids)=['Ġworld', 'ĠWorld', 'Ġmountain', 'Ġgame', 'Ġplanet', 'Ġlatest', 'Ġmap', 'Ġhill', 'Ġlist', 'Ġglobe', 'Ġuniverse', 'Ġtower', 'Ġfood', 'Ġcurve', 'Ġglobal', 'Ġbest', 'Ġboard', 'Ġweb

<div dir=rtl>

[שאלות 6 ו-7 אינן תלויות זו בזו. אם מתקשיםות עם 6, אפשר לוותר על חצי נקודה ולדלג ל-7.]

**6. התשובה לשאלה האחרונה היא לא מה שציפינו עבור תשובה בעלת טוקן אחד. מצאו החל מאיזה מקום בדירוג הטוקנים מקבלים תשובה מהסוג הרצוי (שנה) והאם זו השנה הנכונה. (0.5)**

רמז: יש פונקציה מובנית לטנזורים של פייטורץ' שתקל על פתרון השאלה הזו.

**7. עוד דרך לקבל תשובה נכונה היא להזין את החודש הנכון כהמשך הקלט ולראות מה הפלט הבא. מיצאו את ההסתברות שבהנתן הקלט המקורי (לעיל) המודל יוציא *בדגימה* את החודש הנכון ולאחריו את השנה הנכונה. הראו את החישוב. (0.5)**

<div>

In [137]:
### Q6 + Q7 (more cells possible) ###
obama_sentence = 'Question: When was Barack Obama elected President of the USA? Answer:'
# The Answer should be January 20 2009 (elected) or November 08 2008 (elections)
def q6(sen, true_answer):
  input_tokens = tokenizer.batch_encode_plus([sen], return_tensors='pt')
  outputs = model(**input_tokens)
  logits = outputs[0][0]
  sm_outputs = softmax(logits[-1], dim=0)
  sorted_sm = torch.argsort(sm_outputs, descending=True)
  print(tokenizer.convert_ids_to_tokens(sorted_sm[:20]))

  for i, token in enumerate(tokenizer.convert_ids_to_tokens(sorted_sm)):
      if token[1:] == true_answer:
          print(i, token, sm_outputs[sorted_sm[i]])
          return i, sm_outputs[sorted_sm[i]]
  return -1, 0
  # return best_token_id(sm_outputs)

def q7(sen):
  accumlated_prob = q6(sen, "January")[1] * q6(sen + " January", "2009")[1] * q6(sen + " January 20", "2009")[1]
  print(f"Option 1", accumlated_prob)
  print("\n")
  accumlated_prob = q6(sen, "November")[1] * q6(sen + " November", "2008")[1] * q6(sen + " November 08", "2008")[1]
  print(f"Option 2", accumlated_prob)

print(q6(obama_sentence, "2009")[0])
print(q6(obama_sentence, "2008")[0])
print("\n\n")
q7(obama_sentence)

['ĠJanuary', 'ĠNovember', 'ĠMarch', 'ĠFebruary', 'ĠSeptember', 'ĠOctober', 'ĠJuly', 'ĠIn', 'ĠOn', 'ĠApril', 'ĠHe', 'ĠAugust', 'ĠJune', 'ĠDecember', 'ĠMay', 'ĠThe', 'ĠWhen', 'ĠAfter', 'Ġ9', 'Ġon']
34 Ġ2009 tensor(0.0055, grad_fn=<SelectBackward0>)
34
['ĠJanuary', 'ĠNovember', 'ĠMarch', 'ĠFebruary', 'ĠSeptember', 'ĠOctober', 'ĠJuly', 'ĠIn', 'ĠOn', 'ĠApril', 'ĠHe', 'ĠAugust', 'ĠJune', 'ĠDecember', 'ĠMay', 'ĠThe', 'ĠWhen', 'ĠAfter', 'Ġ9', 'Ġon']
27 Ġ2008 tensor(0.0069, grad_fn=<SelectBackward0>)
27



['ĠJanuary', 'ĠNovember', 'ĠMarch', 'ĠFebruary', 'ĠSeptember', 'ĠOctober', 'ĠJuly', 'ĠIn', 'ĠOn', 'ĠApril', 'ĠHe', 'ĠAugust', 'ĠJune', 'ĠDecember', 'ĠMay', 'ĠThe', 'ĠWhen', 'ĠAfter', 'Ġ9', 'Ġon']
0 ĠJanuary tensor(0.0634, grad_fn=<SelectBackward0>)
['Ġ20', 'Ġ1', 'Ġ25', 'Ġ22', 'Ġ21', 'Ġ27', 'Ġ7', 'Ġ23', 'Ġ15', 'Ġ17', 'Ġ19', 'Ġ8', 'Ġ28', 'Ġ6', 'Ġ2', 'Ġ24', 'Ġ26', 'Ġ9', 'Ġ5', 'Ġ11']
32 Ġ2009 tensor(0.0082, grad_fn=<SelectBackward0>)
[',', 'th', '-', '.', 'Ġ-', 'st', 'ĠâĢĵ', 'Ċ', 'Ġ2009', 'Ġ2012'